In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

2024-01-07 22:41:11.341335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('data.csv', usecols=['type', 'metre', 'text'])
df = df[['metre', 'text']]

In [3]:
df['metre'].unique()

array([1, 2, 0, 4, 3, 6, 7, 5])

In [4]:
train, val, test = np.split(df.sample(frac=1), [int(.8 * len(df)), int(.9 * len(df))])

/home/tesserekt/anaconda3/envs/jasovo_metrum/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
len(train), len(val), len(test)

(1670806, 208851, 208851)

In [6]:
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
  df = dataframe.copy()
  labels = df.pop('metre')
  df = df["text"]
  ds = tf.data.Dataset.from_tensor_slices((df, labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)

  return ds

In [7]:
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)
print(list(test_data.take(1)))

[(<tf.Tensor: shape=(1024,), dtype=string, numpy=
array([b'"dru hy dri ma vac slav kral"', b'"ba i ve le slav ny whist"',
       b'"ach kte rou mu zu bu du vzy va ti"', ...,
       b'"ja ko zbe si ly se ru mu vrh nul"',
       b'"pod or ly va se a jsi za se hlu pak"',
       b'"a hned co se sta lo ro zu mel"'], dtype=object)>, <tf.Tensor: shape=(1024,), dtype=int64, numpy=array([1, 1, 0, ..., 1, 0, 1])>)]


2024-01-07 22:41:26.634071: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-01-07 22:41:26.802610: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [208851]
	 [[{{node Placeholder/_1}}]]
2024-01-07 22:41:26.802761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [208851]
	 [[{{node Placeholder/_1}}]]


In [8]:
encoder = tf.keras.layers.TextVectorization()
encoder.adapt(train_data.map(lambda text, label: text))

2024-01-07 22:41:32.739688: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1670806]
	 [[{{node Placeholder/_1}}]]
2024-01-07 22:41:32.739817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1670806]
	 [[{{node Placeholder/_1}}]]


In [9]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]
len(vocab)

13550

In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=32,
        mask_zero=True
    ),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(8, activation='sigmoid')
])

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [12]:
history = model.fit(train_data, epochs=30, validation_data=valid_data)

Epoch 1/30


2024-01-07 22:41:47.950245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1670806]
	 [[{{node Placeholder/_1}}]]
2024-01-07 22:41:47.950467: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1670806]
	 [[{{node Placeholder/_0}}]]


1630/1632 [============================>.] - ETA: 0s - loss: 0.8681 - accuracy: 0.6737

2024-01-07 22:42:28.855732: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [208851]
	 [[{{node Placeholder/_0}}]]
2024-01-07 22:42:28.855872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [208851]
	 [[{{node Placeholder/_1}}]]


1632/1632 [==============================] - 43s 25ms/step - loss: 0.8679 - accuracy: 0.6738 - val_loss: 0.6697 - val_accuracy: 0.7703
Epoch 2/30
1632/1632 [==============================] - 39s 24ms/step - loss: 0.6596 - accuracy: 0.7888 - val_loss: 0.5974 - val_accuracy: 0.8094
Epoch 3/30
1632/1632 [==============================] - 39s 23ms/step - loss: 0.5957 - accuracy: 0.8169 - val_loss: 0.5388 - val_accuracy: 0.8323
Epoch 4/30
1632/1632 [==============================] - 39s 23ms/step - loss: 0.5101 - accuracy: 0.8505 - val_loss: 0.4328 - val_accuracy: 0.8735
Epoch 5/30
1632/1632 [==============================] - 41s 25ms/step - loss: 0.4247 - accuracy: 0.8816 - val_loss: 0.3825 - val_accuracy: 0.8912
Epoch 6/30
 843/1632 [==============>...............] - ETA: 17s - loss: 0.3925 - accuracy: 0.8912

KeyboardInterrupt: 

In [31]:
model.evaluate(test_data)

204/204 [==============================] - 3s 12ms/step - loss: 0.2969 - accuracy: 0.9192


[0.29694095253944397, 0.9191768169403076]

In [33]:
model.save('metre_detector.tf')

2024-01-07 15:40:48.345536: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-07 15:40:48.346253: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-07 15:40:48.347016: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: metre_detector.tf/assets


INFO:tensorflow:Assets written to: metre_detector.tf/assets


In [5]:
model = tf.keras.models.load_model('metre_detector.tf')

2024-01-07 18:14:16.332530: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-01-07 18:14:16.536209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2024-01-07 18:14:16.536253: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2024-01-07 18:14:16.5

In [12]:
np.argmax(model.predict(test_data)[0])

2024-01-07 18:14:54.615375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [208851]
	 [[{{node Placeholder/_1}}]]
2024-01-07 18:14:54.615499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [208851]
	 [[{{node Placeholder/_1}}]]


204/204 [==============================] - 4s 15ms/step


0